## Accuracy using RRWicks Approach

In [7]:
from pathlib import Path

In [8]:
base_dir = Path('../../../')
bonito_taiyaki = base_dir/"bonito_taiyaki.fasta"
rrwick_dir = base_dir/"August-2019-consensus-accuracy-update/"

## Generate Fasta Reference

In [12]:
import h5py, glob

In [86]:
from ont_fast5_api.fast5_interface import get_fast5_file
import numpy as np

In [55]:
reads_dir = base_dir/"taiyaki_walkthrough/reads/"
fast5s = glob.glob(f"{reads_dir}/*")

In [102]:
def get_read_id(filename):
    """
    Get the raw signal and read id from the fast5 files
    """
    with get_fast5_file(filename, 'r') as f5_fh:
        for read in f5_fh.get_reads():
            yield read.read_id

In [104]:
def get_single_from_generator(generator):
    for i in generator:
        return i

In [106]:
get_single_from_generator(get_read_id(fast5s[0]))

'0c547b2d-e89d-4216-93f0-512d4d1da710'

In [56]:
f0 = h5py.File(fast5s[0], 'r')

In [84]:
f0.global_key

AttributeError: 'File' object has no attribute 'global_key'

In [22]:
bonito_td = h5py.File(base_dir/"bonito/bonito/data/bonito-training-data.hdf5", 'r')

In [112]:
reads = bonito_td['Reads']
rkeys = list(reads.keys())
rkeys

['00001adf-3ecc-49c3-824f-68f88e475ddf',
 '00003587-d7a4-4f1a-a5d5-a69afb4a2216',
 '0000d7d8-6954-4018-9f08-74e968d50107',
 '00014b4a-a831-4588-a1d1-5dfe2fb47a47',
 '00015fc0-24c1-45ee-89f8-132def672e1d',
 '0001da81-5802-4d9b-b157-1da6135e26d8',
 '0002a240-dd4f-4e55-a8b5-98d08acc7f3f',
 '0004c902-d7fe-40cd-8f13-655f8c37d310',
 '00057315-9c44-4b83-9b0a-61df8ebbc2ce',
 '0006361a-79e8-4cd8-956e-663ac3594bcb',
 '0007f44a-d152-4230-acd0-c1c1a44f5d15',
 '000af5ca-e0d5-4e1c-af71-d6fde105349c',
 '000b036d-78c1-4c80-a696-bd9987dd1640',
 '000ba69f-1203-4e84-b91e-8859619d3736',
 '000be180-d594-44e2-afea-547310612655',
 '000bfdc8-08b6-4aab-b008-ef6ad2a03e00',
 '000c6d01-36df-4373-b39d-1deb76a1cc6f',
 '000d04f3-c8e0-4d7e-8c66-129034110433',
 '000d9aef-d41a-4d84-a833-54e5f07a567f',
 '000d9b31-5ae3-4fe2-a15f-a7efeba46daa',
 '000f047d-61cd-4ec4-9767-261c45b373cf',
 '0011f20c-c27c-485e-ae65-ca4a3c112f6b',
 '001201d4-9778-429d-9d16-3da067f77fc3',
 '001206b2-906a-4b64-a667-f8ed3e586d17',
 '0012e94f-8366-

In [31]:
r0 = reads[rkeys[0]]

In [77]:
r0.keys()

<KeysViewHDF5 ['Dacs', 'Ref_to_signal', 'Reference']>

In [70]:
def srr(r):
    return r.name, r['Dacs'], r['Ref_to_signal'], r['Reference']

In [71]:
ref_name, squiggle, ref2sig, ref = srr(r0)

In [72]:
def stringify_ref(ref, alphabet={0: 'A', 1: 'C', 2: 'G', 3: 'T'}) -> str:
    to_letter = lambda x: alphabet[x]
    concat = lambda xs: "".join(xs)
    map_l = lambda f, x: list(map(f, x))
    
    return concat(map_l(to_letter, ref))    

In [140]:
def save_ref_string_to_fasta(file_name, ref_name: str, ref_str: str):
    with open(file_name, 'w') as fh:
        fh.write(f'> {ref_name}\n')
        fh.write(ref_str)        

In [74]:
ref_str = stringify_ref(ref)

In [75]:
save_ref_string_to_fasta(base_dir/"mynew.fasta", ref_name, ref_str)

In [2]:
! cd /usr/local/bin; ls

change_mofed_version.sh  cudaCheck    nsys
checkSMVER.sh		 deviceQuery  nvidia_entrypoint.sh


In [48]:
! cd ../../../minimap2-2.17_x64-linux/bin; pwd

/user/student.aau.dk/kbargs15/minimap2-2.17_x64-linux/bin


In [130]:
bonito_fn = base_dir/"bonito/bonito/data/bonito-training-data.hdf5"

In [115]:
# Get names of all signals

 ##read_ids = [get_single_from_generator(get_read_id(fast5s[i])) for i in range(len(fast5s))]
read_ids = list(bonito_td['Reads'].keys())
# Get references for all signals
refs = bonito_td['Reads']
# Save >name + ref for all signals to single file

In [118]:
rkeys = list(bonito_td['Reads'].keys())

In [128]:
def get_ids_and_refs(hdf5_fn):
    with h5py.File(hdf5_fn, 'r') as hdf5:
        reads = hdf5['Reads']
        ids = list(reads.keys())

        for id in ids:
            ref_str = stringify_ref(reads[id]['Reference'])
            yield id, ref_str

In [145]:
from tqdm import tqdm

In [129]:
get_ids_and_refs()

<generator object get_ids_and_refs at 0x7f94433993d0>


In [146]:
for id, ref_str in tqdm(get_ids_and_refs(bonito_fn)):
    save_ref_string_to_fasta(base_dir/f"bonito_fasta/{id}.fasta", id, ref_str)

15it [00:06,  2.10it/s]

KeyboardInterrupt: 

In [ ]:
python3 scripts/chop_up_assembly.py $1 100000 > pieces.fasta
minimap2 -x asm5 -t 8 -c hybrid_reference.fasta pieces.fasta > pieces.paf
python3 scripts/read_length_identity.py pieces.fasta pieces.paf > pieces.data
printf $1"\t"
python3 scripts/medians.py pieces.data
rm pieces.*

In [147]:
def load_fasta(fasta_filename):
    fasta_seqs = []
    with open(fasta_filename, 'rt') as fasta_file:
        name = ''
        sequence = ''
        for line in fasta_file:
            line = line.strip()
            if not line:
                continue
            if line[0] == '>':  # Header line = start of new contig
                if name:
                    fasta_seqs.append((name.split()[0], sequence, name))
                    sequence = ''
                name = line[1:]
            else:
                sequence += line
        if name:
            fasta_seqs.append((name.split()[0], sequence, name))
    return fasta_seqs

In [148]:
load_fasta(rrwick_dir/"scripts/pred.fasta")

[('00001adf-3ecc-49c3-824f-68f88e475ddf',
  'GTGAACTGTGTAGAGAGTTGTGCAAAATAAATGCATCTGATACTCCTGATTCACTGCTCATGAGAGGCAAGGAGTTTAGTGACTTTATACCTAATACCTTTGACCATATGTGGAGAACCAAGGAATATAATGAAGCTAGTTGGTTGCTCCTAAGTTCACTAATCAAAGTGATGAAAACAAAAAAATGATGAACTCAGGGATTCTATCTCCCAGCTCCAGAACCACATTCTGAGCCTCAAATCTTCTAAGATTGCCCTGGGTTCGAGTCTTATCTCCTGTAGAGAAAGAGCTGAAATTGCGGAAAATCAGACACAAGCTCTTATCATGTGAGTGGCTGACCTGCAACGAAAGGTACGTGCACAGCCTTGCCAGGTGTTTACTGTTAAAGTGACAGCATTGATTGAAAAAGAATGGTACCCTGCAACTTGGAATGGGGCCATGCGGGAGGACCCTGATAAGGCTGGGGGCACTGAGCTTCTAAATTCTGATGAATCTTTTTTACCAGAGGAAACAGGCAACAGAGGGGAGGCAACATCCCCTCCTCCACCTGCGCTGCCATCAGGCTTTTCACCTTTGTCTGAGGAGTTTAACCCTGCACTGCCTAAGGCATCAGCGATGACCTCCCTTGAGGCAGTTGCCAGGAAAGACGATGTTGATTCTCCTAAGGACCCACCCTCAATACCCCTGTTTGCTTATAGTTATAGAACTAGACTAAAGTCCCGGCAGGCCCCTTAGAGATGAGGTTCAGAATGTGACCCACAAGGAGGTGCGCTACACTCCAAAAGAACTGCTTGAGTTTTCTAATTTATATAAGCAGAAATCTGGAGAACAGGCATGGGAATGGATATTAAGGGTGTGGGCTAATGGTAGAAGGAACATAAAGTTGGATCAGGCTGAATTGATTGATTTGGGCCCACTAAGCAGGAATTCTGCATTTAATGTTGCAGCTTGGAGA

In [150]:
"a ok".split()

['a', 'ok']

In [152]:
import jkbc.utils.accuracy as acc

In [153]:
!ls

CTCLoss_Example.ipynb			  UnderstandingRNNs.ipynb
Chiron-binary.ipynb			  UnderstandingRNNs2.ipynb
JKBC-CNN.ipynb				  acc_rrwick.ipynb
JKBC-Simple.ipynb			  p9
KD-MNIST.ipynb				  simple_rnn.ipynb
PyTorch_Basics_and_Reshape_meaning.ipynb  train_like_bonito
Train_Like_Bonito.ipynb


In [155]:
acc.get_accuracy("Train_Like_Bonito.ipynb", "ok")